<a href="https://colab.research.google.com/github/BaidyaSubhadip/turantlo_task/blob/main/Disease_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.head(5)

,Disease_Name,Symptom_List
0,Aarskog syndrome,"['Mildly sunken chest (pectus excavatum) ', ' ..."
1,Aase syndrome,"['Narrow shoulders ', ' Droopy eyelids ', ' Ab..."
2,Abdominal aortic aneurysm,"['Passing out. ', ' Dizziness. ', ' Pain in th..."
3,Abnormal uterine bleeding,"['Tenderness and dryness of the vagina ', ' He..."
4,ABO incompatibility,"['Feeling of ""impending doom"" ', ' Chest pain ..."


In [4]:
unique_symptoms = []
for symptoms in data['Symptom_List']:
  symptom_list = eval(symptoms)
  unique_symptoms.extend(symptom_list)
unique_symptoms = sorted(set(unique_symptoms))
#print(unique_symptoms)

In [5]:
print(len(unique_symptoms))

9681


In [6]:
import spacy

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
def tokenize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

In [9]:
def calculate_similarity(input_tokens, symptom_tokens):
    common_tokens = set(input_tokens) & set(symptom_tokens)
    return len(common_tokens) / (len(set(input_tokens)) + len(set(symptom_tokens)) - len(common_tokens))

In [12]:
def find_closest_symptoms(input_text, symptoms_list, top_n=5):
    input_tokens = tokenize(input_text)
    similarities = []
    for symptom in symptoms_list:
        symptom_tokens = tokenize(symptom)
        similarity = calculate_similarity(input_tokens, symptom_tokens)
        similarities.append((symptom, similarity))


    closest_symptoms = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]
    return closest_symptoms


In [13]:
def process_paragraph(input_text, symptoms_list, top_n=5):
    doc = nlp(input_text)
    all_symptoms = []
    for sent in doc.sents:
        closest_symptoms = find_closest_symptoms(sent.text, symptoms_list, top_n=top_n)
        all_symptoms.extend(closest_symptoms)

    unique_symptoms = {symptom: score for symptom, score in all_symptoms}
    sorted_symptoms = sorted(unique_symptoms.items(), key=lambda x: x[1], reverse=True)
    return sorted_symptoms[:top_n]

In [14]:
input_paragraph = """
I've been noticing some unusual physical characteristics in myself, including a mildly sunken chest and undescended testicles.
Additionally, I've been experiencing mild to moderate cognitive problems.
I've also observed some distinct facial features, such as a belly button that sticks out, a top portion of the ear that's folded over slightly, and a small nose with nostrils tipped forward.
I've also had delayed teeth and a scrotum that surrounds my penis, giving it a "shawl scrotum" appearance.
"""

closest_symptoms = process_paragraph(input_paragraph, unique_symptoms, top_n=5)
print("Closest Symptoms:")
for symptom, similarity in closest_symptoms:
    print(f"{symptom}: {similarity:.4f}")

Closest Symptoms:
 Mild to moderate cognitive problems : 0.5000
 Scrotum surrounds the penis (shawl scrotum) : 0.3636
 Small nose with nostrils tipped forward : 0.2778
Mildly sunken chest (pectus excavatum) : 0.2308
 Mild-to-moderate dementia : 0.2222
